# Customer Service Data Clustering

In [ ]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import os
import re

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from MulticoreTSNE import MulticoreTSNE as TSNE

import pickle

from tqdm import tqdm

from sklearn.cluster import DBSCAN

### Configuration

In [ ]:
path = os.path.join("..","data","./newdata_clean.xlsx")
n_cpu = 15
batch_size = 10000
max_k = 40
max_features = 256  # only consider the top max_features ordered by term frequency across the corpus.
loadpath = "processed_data_lda_wo_html"
#loadpath = "processed_data_not_rmsw"

In [ ]:
with open(loadpath, "rb") as f:
    output = pickle.load(f)
clean_data = output["clean_data"]
reduced_data = output["reduced_data"]
token_data = output["token_data"]

In [ ]:
def find_optimal_clusters(data, max_k, batch_size):
    iters = range(2, max_k+1, 2)
    
    sse = []
    bar = tqdm(iters)
    for k in bar:
        sse.append(MiniBatchKMeans(n_clusters=k, batch_size=batch_size, random_state=20).fit(data).inertia_)
        bar.set_description('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')

In [ ]:
def cluster_and_plot(data, cluster_algo, n_clusters, eps, calculate_portion, draw_portion):
    n_data = data.shape[0]
    print("Input Data shape:", data.shape)
    
    # random sample (n_sample) points
    np.random.seed(5)
    n_sample = n_data // calculate_portion
    print("Only calculate {} data points".format(n_sample))
    sample_items = np.random.choice(range(n_data), size=n_sample, replace=False)
    
    
    if cluster_algo == "kmeans":
        labels = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, random_state=20).fit_predict(data)
        npdata = data[sample_items,:]
        label_subset = labels[sample_items]
    elif cluster_algo == "dbscan":
        npdata = data[sample_items]
        labels = DBSCAN(eps=eps, n_jobs=n_cpu).fit_predict(npdata)
        label_subset = labels
        print("Number of noise data: {}".format(len(label_subset[label_subset == -1])))
    
    # number of clusters
    max_label = max(labels) 
    print("Number of clusters: {}".format(max_label + 1))
    pca = PCA(n_components=2, whiten=True).fit_transform(npdata)
    print("PCA done")
    tsne_pca = TSNE(n_jobs=n_cpu, n_iter=5000).fit_transform(PCA(n_components=(max_features // 2)).fit_transform(npdata))
    print("tsne_pca done")
    tsne = TSNE(n_jobs=n_cpu, n_iter=5000).fit_transform(npdata)
    print("tsne30 done")
    tsne_per50 = TSNE(n_jobs=n_cpu, perplexity=50, n_iter=5000).fit_transform(npdata)
    print("tsne50 done")
    
    # draw only (n_draw) points
    np.random.seed(5)
    n_draw = n_sample // draw_portion
    print("Only draw {} data points".format(n_draw))
    idx = np.random.choice(range(pca.shape[0]), size=n_draw, replace=False)
    
    # draw scatter
    if cluster_algo == "kmeans":
        start_idx = 0
    else:
        start_idx = -1
    f, ax = plt.subplots(2, 2, figsize=(13, 10))
    for i in range(start_idx,max_label + 1):
        sub_idx = idx[label_subset[idx] == i]
        #print("{} points in group {}".format(len(sub_idx),i))
        label_subset_color = np.array([cm.hsv(i/ ( max_label + 1)) for i in label_subset[sub_idx]])
        ax[0,0].scatter(pca[sub_idx, 0], pca[sub_idx, 1])   # , c=label_subset_color
        ax[0,0].set_title('PCA Cluster Plot')
        
        ax[1,0].set_title('t-SNE & PCA Cluster Plot')
        ax[1,0].scatter(tsne_pca[sub_idx, 0], tsne_pca[sub_idx, 1], label="Group {} | {}".format(i,len(sub_idx))) # , c=label_subset_color
        
        ax[0,1].set_title('t-SNE Cluster Plot')
        ax[0,1].scatter(tsne[sub_idx, 0], tsne[sub_idx, 1]) # , c=label_subset_color
        
        ax[1,1].set_title('t-SNE Cluster Plot (Perplexity 50)')
        ax[1,1].scatter(tsne_per50[sub_idx, 0], tsne_per50[sub_idx, 1]) # , c=label_subset_color
    f.legend() # plot only one legend
    
    if cluster_algo == "kmeans":
        return data, labels
    else:
        return npdata, labels

## TF-IDF Clustering

In [ ]:
print("max_feature",max_features)
tfidf = TfidfVectorizer(
    min_df = 0.001,
    max_df = 0.95,
    max_features = max_features,
    stop_words = 'english'
)
tfidf.fit(clean_data)
text = tfidf.transform(clean_data)

In [ ]:
find_optimal_clusters(text, max_k, batch_size)

### Clustering by Kmeans

In [ ]:
sample_text, sample_text_labels = cluster_and_plot(text, "kmeans", 10, None, 500, 10)

### Clustering by DBSCAN

In [ ]:
cluster_and_plot(text, "dbscan", None, 0.999, 50, 10)

### Get keywords of each TF-IDF cluster
作法是將每個 cluster 的前幾高 tfidf 字取出來，作為這個 cluster 的代表字。但是效果不好，且很多 cluster 都有相同常見的字(ex: download)

In [ ]:
def get_top_keywords(data, text, clusters, labels, n_terms):
    data = np.array(data)
    clusters = np.array(clusters)
    text_feature = text.toarray()
    labels = np.array(labels)
    group = pd.DataFrame(text.todense()).groupby(clusters)
    #print("Data point in each group:\n", group.size())
    df = group.mean() # [(clusters) rows x (feature) columns]

    for i,r in df.iterrows():
        sub_text = data[clusters == i]
        sub_text_list = sub_text
        sub_text_feature = text_feature[clusters == i]
        #print("sub_text: {} | sub_text_feature: {}".format(sub_text.shape,sub_text_feature.shape))

        dist = np.linalg.norm(sub_text_feature - np.array(r), axis=1)

        #print("Euclidean distance:", dist.shape, dist)
        
        print('\nCluster {}'.format(i))
        print(', '.join([labels[t] for t in np.argsort(r)[-n_terms:]]))
        #print(','.join([str(r[t]) for t in np.argsort(r)[-n_terms:]]))
        print('\n'.join([re.sub(' +', ' ',_filter(sub_text_list[t])) + " | " + str(dist[t]) for t in np.argsort(dist)[-2:]]))
            
get_top_keywords(reduced_data, sample_text, sample_text_labels, tfidf.get_feature_names(), 10)

## Doc2vec Clustering

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
tagged_data = [TaggedDocument(words=tokens, tags=[str(i)]) for i, tokens in enumerate(token_data)]

max_epochs = 100
alpha = 0.025

model = Doc2Vec(vector_size=max_features,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                workers=n_cpu,
                dm=1)
  
model.build_vocab(tagged_data)
bar = tqdm(range(max_epochs))
for epoch in bar:
    bar.set_description('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [ ]:
#model= Doc2Vec.load("d2v.model")
model= Doc2Vec.load("enwiki_dbow/doc2vec.bin")
'''
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)


# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])
'''

n_reduced_data = len(reduced_data)
docvec = []
for i in tqdm(range(n_reduced_data)):
    #docvec.append(model.docvecs[str(i)])
    docvec.append(model.infer_vector(token_data[i]))
docvec = np.array(docvec)
print("Number of data: {}".format(n_reduced_data))

In [ ]:
find_optimal_clusters(docvec, max_k, batch_size)

### Clustering by Kmeans

In [ ]:
_ = cluster_and_plot(docvec, "kmeans", 10, None, 50, 10)

### Clustering by DBSCAN

In [ ]:
_ = cluster_and_plot(docvec, "dbscan", None, 2.1, 50, 10)

### Reference
* [Clustering documents with TFIDF and KMeans](https://www.kaggle.com/jbencina/clustering-documents-with-tfidf-and-kmeans)
* [Analyzing tf-idf results in scikit-learn](https://buhrmann.github.io/tfidf-analysis.html)

## Bert Sentence Encoding Clustering

1. 要先把 bert server 開起來
    `gpu0 bert-serving-start -model_dir uncased_L-12_H-768_A-12 -num_worker 4  -port 1355 -max_seq_len 40 -device_map 0`
2. 直到出現 `all set, ready to serve request!` 才可以 run client 的 command
Note:
1. 如果是 run 大的 model 會 load 不進去 GPU 中，跑出 OOM
2. 如果系統記憶體不夠則不會出現 all set, ready to serve request!，CPU 版來說，最多只能 -num_worker 2

In [ ]:
from bert_serving.client import BertClient
bc = BertClient(port=1355)
print("Start predicting")
bert_output = bc.encode(clean_data)
bert_output[:10]

In [ ]:
bert_data_path = "bert_base.pkl"

In [ ]:
bert_data = {
    "clean_data": clean_data,
    "reduced_data": reduced_data,
    "token_data": token_data,
    "bert_data": bert_output
}
with open(bert_data_path, "wb") as f:
    pickle.dump(bert_data, f)

In [ ]:
with open(bert_data_path, "rb") as f:
    bert_data = pickle.load(f)
clean_data = bert_data["clean_data"]
reduced_data = bert_data["reduced_data"]
token_data = bert_data["token_data"]
bert_output = bert_data["bert_data"]

In [ ]:
find_optimal_clusters(bert_output, max_k, batch_size)

### Clustering by Kmeans

In [ ]:
_ = cluster_and_plot(bert_output, "kmeans", 10, None, 50, 10)

### Clustering by DBSCAN

In [ ]:
_ = cluster_and_plot(bert_output, "dbscan", None, 5.95, 50, 10)